# Crop Productivity Analysis 

## Literature Review: Remote Sensing for Agricultural Yield Prediction

The relationship between crop yields and the Enhanced Vegetation Index (EVI), a vegetation index derived from satellite imagery that is particularly effective in monitoring vegetation health and productivity, has been extensively studied. Johnson (2016) provides a comprehensive overview of the use of EVI in agricultural yield prediction, highlighting its advantages over other indices like the Normalized Difference Vegetation Index (NDVI). The study concluded that EVI was among the best overall performer in predicting crop yields, showing the highest correlation in five out of nine crops studied, excluding rice. Numerous studies have successfully utilized EVI to predict yields of various crops, including maize, soybeans, and rice. For instance, Bolton and Friedl (2013) created  a linear model using EVI to forecast soybean and maize yields in the Central United States. Their findings demonstrate that EVI exhibits a stronger correlation with maize yield and yield anomalies than other vegetation indices, and incorporating phenological data significantly enhances the model’s performance. A similar finding by Ji et al. (2022) showed that their model framework based on EVI and crop phenology can improve the accuracy of yield predictions of corn and soybean in the United States. Furthermore, Son et al. (2013) used MODIS EVI data for rice yield prediction in the Vietnamese Mekong Delta, reporting high correlation coefficients (R2 of 0.70 for spring-winter and 0.74 for autumn-summer in 2013, and 0.62-0.71 and 0.4-0.56 in 2014, respectively). 

### Satellite and Ground-Based Approaches

Compared to ground-based methods, such as field interviews and crop cuttings, Remote sensing approach has several advantages on estimating crop yields:

- Timeliness and Rapid Assessment: Remote sensing offers real-time or near real-time assessment of crop conditions and potential yields, which is crucial for situations like natural disasters or conflicts (Deininger et al. 2023; Doraiswamy et al. 2003).
- Extensive Spatial Coverage and Granularity: Satellite imagery can consistently cover large geographic areas, providing granular, field, and village-level data that can be aggregated to higher administrative levels, such as districts or provinces (Azzari, Jain, and Lobell 2017; Becker-Reshef et al. 2010).
- Cost-Effectiveness: Remote sensing is generally more cost-effective for collecting agricultural data over large areas compared to ground-based methods (Johnson 2016; Rahman et al. 2009).
- Accessibility in Difficult or Dangerous Areas: Remote sensing provides a reliable option for data collection in zones that are difficult, dangerous, or inaccessible for in-situ surveys, such as conflict areas (Jaafar et al. 2015).
However, remote sensing also has several limitations:
- Spatial Resolution Limitations: While modern sensors offer higher resolutions, coarse resolution data may not accurately reflect ground situations for small agricultural clusters, small parcels, or low-intensity agriculture (Kibret, Marohn, and Cadisch 2020).
- Significant Computational Demands: Analyzing high-resolution imagery requires massive amounts of computational power and storage, which can be expensive and time-consuming (Petersen 2018).
- Interpretability and Bias Concerns: Satellite-derived metrics may not fully capture all determinants of crop production and may not quantitatively interpret crop growth status (Wu et al. 2023)

While remote sensing offers timely, broad-scale, and cost-effective monitoring, particularly in data-scarce or inaccessible regions, it faces challenges with resolution limitations, computational demands, and potential biases in interpreting crop conditions.

### EVI and Other Vegetation Indices
Within the remote sensing approach itself, multiple vegetation indices have been utilized to enhance the accuracy of yield predictions. Compared to other vegetation indices, EVI offers several advantages over other vegetation indices:
- Reduced saturation: EVI does not saturate as quickly as NDVI at higher crop
leaf area or in areas with large amounts of biomass, providing improved sensitivity in dense vegetation conditions (Huete et al. 2002). This is crucial as high saturation in indicators like NDVI can lead to unreliable yield estimates (Son et al. 2013).
- Atmospheric and Soil Correction: EVI incorporates a soil adjustment factor and corrections for the red band due to aerosol scattering, making it more resistant to atmospheric influences and soil background noise compared to NDVI (Jurečka et al. 2018).

On the other hand, EVI also has some limitations compared to other vegetation indices:

- Temporal Resolution and Latency: While EVI is disseminated at 250m resolution, it may only be available at 16-day time steps, compared to NDVI’s 8-day availability, which could introduce latency issues for real-time monitoring (Johnson 2016)
- Mixed Pixel Problems: Coarse spatial resolution of MODIS EVI (250m or 500m) can limit performance in regions with small, fragmented parcels (Kibret, Marohn, and Cadisch 2020).

### Climate influenced on EVI

EVI values are influenced by several environmental factors. EVI is sensitive to rainfall patterns, as the variations of phenological stages of crops, such as the timing of planting and harvesting, are closely linked to rainfall (Kibret, Marohn, and Cadisch 2020). Additionally, while EVI is designed to be resistant to atmospheric aerosols, cloud contamination, particularly during wet seasons, can affect vegetation greenness signals and lower the accuracy of yield predictions (Son et al. 2013).

### Modelling Approaches of EVI

Several modeling approaches have been successfully employed with EVI data for crop yield prediction. Statistical models, particularly linear and quadratic regression, are widely used due to their fewer data requirements and assumptions compared to biophysical models (Ji et al. 2022). Linear regression models are frequently applied, with EVI generally performing well and showing more linear relationships with yields than NDVI for certain crops (Tiruneh et al. 2023). Machine learning approaches have also proven effective, with Kibret, Marohn, and Cadisch (2020) applied Random Forest algorithm to MODIS EVI time series for agricultural land use classification and cropping system identification. Pham et al. (2022) demonstrated that integrating Principal Component Analysis (PCA) with machine learning methods (PCA-ML) on VCI data effectively addresses spatial variability and redundant data issues, enhancing prediction accuracy by up to 45% in rice yield forecasting for Vietnam.

## Data and Assumptions

### Data

The following datasets are utilized in this analysis for calculating and mapping crop productivity over the past years:

1. **Dynamic World Dataset**:
   - **Source**: [Dynamic World - Google and the World Resources Institute (WRI)](https://dynamicworld.app/) 
   - **Description**: The Dynamic World dataset provides a near real-time, high-resolution (10-meter) global land cover classification. It is derived from Sentinel-2 imagery and utilizes machine learning models to classify land cover into nine distinct classes, including water, trees, grass, crops, built areas, bare ground, shrubs, flooded vegetation, and snow/ice. The dataset offers data with minimal latency, enabling near-immediate analysis and decision-making.
   - **Spatial Resolution**: 10 meters.
   - **Temporal Coverage**: Data is available since mid-2015, updated continuously as Sentinel-2 imagery becomes available capturing near Real-time.

2. **MODIS Dataset**:
   - **Source:** NASA's Moderate Resolution Imaging Spectroradiometer [MODIS](https://modis.gsfc.nasa.gov/data/) on Terra and Aqua satellites.
   - **Description:** The MODIS dataset provides a wide range of data products, including land surface temperature, vegetation indices, and land cover classifications. It is widely used for monitoring and modeling land surface processes.
   - **Spatial Resolution:** 250 meters.
   - **Temporal Coverage:** Data is available from 2000 to the present, with daily to 16-day composite products.

3. **GDP and Agricultural Data**:
   - **Source**: Official government statistics and international databases.
   - **Description**: This dataset includes regional GDP figures, agricultural output, export data, and crop production statistics. It is used to analyze the economic impact of agricultural productivity.
   - **Spatial Resolution**: Varies by dataset; typically available at national and subnational levels.
   - **Temporal Coverage**: Varies by dataset; typically available annually or quarterly.

4. **Administrative Boundaries**:
   - **Source**: MIMU (Myanmar Information Management Unit)
   - **Description**: This dataset provides the administrative boundaries for Myanmar at various levels (national, state/region, district, township). It is used for spatial aggregation and analysis of crop productivity and economic data.
   - **Spatial Resolution**: Varies by administrative level.

### Assumptions and Methodological Notes

This analysis is based on the following assumptions and methodological considerations:

#### Temporal Definitions and Aggregations

1. **Fiscal Year Definition**: The first quarter (Q1) of each fiscal year starts in April and ends in March of the following year. This aligns with Myanmar's fiscal calendar.

2. **Quarterly Aggregations**: Monthly data are aggregated into quarters (Q1: April-June, Q2: July-September, Q3: October-December, Q4: January-March).

#### Methodological Assumptions

1. **Spatial Aggregation**: Regional GDP values are based on single-year reference data and are assumed to maintain consistent spatial distributions across all years in the analysis period.

2. **EVI Processing**: 
   - EVI values are median-aggregated within administrative boundaries to reduce noise
   - Time series preprocessing follows TIMESAT methodology: outlier removal, interpolation, and Savitzky-Golay smoothing
   - Seasonality parameters (start of season, middle of season, end of season) are extracted using a 20% threshold of maximum EVI

3. **Cropland Identification**: Pixels are classified as cropland based on Dynamic World's machine learning classification. Classification accuracy varies by region and land cover type, with potential confusion between cropland and similar land covers.R

## Setup and Data Preparation

### Environment Setup

In [1]:
from pathlib import Path
import warnings

import altair as alt
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from stargazer.stargazer import Stargazer
from sklearn.metrics import (
    mean_absolute_error,
    root_mean_squared_error,
    mean_absolute_percentage_error,
)

warnings.filterwarnings("ignore")

### Helper functions

In [2]:
def find_project_root(marker_file: str = "pyproject.toml"):
    for parent in Path().cwd().parents:
        if (parent / marker_file).exists():
            return parent
    raise FileNotFoundError(f"No project root found with marker file: {marker_file}")


PROJECT_ROOT = Path().cwd().parent.parent
DATA_PATH = PROJECT_ROOT / "data"
EVI_PATH = DATA_PATH / "EVI and Crop Land" / "EVI 2010-2025"
CROPLAND_PATH = DATA_PATH / "EVI and Crop Land" / "Crop Land"
BOUNDARIES_PATH = DATA_PATH / "Shapefiles"

In [3]:
def clean_names(df: pd.DataFrame) -> pd.DataFrame:
    """Standardize column names."""
    df = df.rename(columns=lambda col: col.strip().lower().replace(" ", "_"))
    return df


def preprocess_evi(evi_file: str | Path) -> pd.DataFrame:
    """Preprocess EVI CSV file."""
    evi_df = pd.read_csv(evi_file)

    metadata_cols = [
        col for col in evi_df.columns if "EVI" not in col and "system:index" not in col
    ]
    evi_df = (
        evi_df.rename(columns=lambda col: col[-14:] if col.endswith("_EVI") else col)
        .drop(columns=["system:index"])
        .melt(
            id_vars=metadata_cols,
            var_name="band_date",
            value_name="EVI",
        )
        .assign(
            date=lambda df: pd.to_datetime(
                df["band_date"].str.extract(r"(\d{4}_\d{2}_\d{2})")[0],
                format="%Y_%m_%d",
            ),
        )
    )
    return evi_df

In [4]:
gdp_quarterly_raw = pd.read_excel(
    DATA_PATH / "GDP" / "Quarterly GDP per Sector.xlsx",
    sheet_name=1,
    skiprows=1,
    skipfooter=1,
)
gdp_adm1_raw = pd.read_excel(
    DATA_PATH / "GDP" / "Quarterly GDP per Sector.xlsx", sheet_name=0, skiprows=1
).rename(columns={"Unnamed: 0": "ADM1_NAME"})

evi_adm0_raw = (
    pd.concat(
        [
            preprocess_evi(
                DATA_PATH
                / "EVI and Crop Land"
                / "EVI 2010-2025"
                / "Admin level 0"
                / "MIMU"
                / f"myanmar_adm0_evi_stats_{year}.csv"
            )
            for year in range(2010, 2026)
        ],
    )
    .drop(columns=[".geo"])
    .sort_values(["date"])
    .reset_index(drop=True)
    .set_index("date")
)
evi_adm1_raw = (
    pd.concat(
        [
            preprocess_evi(
                DATA_PATH
                / "EVI and Crop Land"
                / "EVI 2010-2025"
                / "Admin level 1"
                / "MIMU"
                / f"myanmar_adm1_evi_stats_{year}_batch{batch}.csv"
            )
            for year in range(2010, 2026)
            for batch in range(1, 4)
        ],
    )
    .drop(columns=[".geo"])
    .sort_values(["ST", "date"])
    .reset_index(drop=True)
    .rename(columns={"ST": "adm1_name"})
    .set_index("date")
)

evi_adm0 = (
    evi_adm0_raw.groupby(pd.Grouper(freq="QS")).agg({"EVI": "median"}).pipe(clean_names)
)

evi_adm1 = (
    evi_adm1_raw.groupby(["adm1_name", pd.Grouper(freq="QS")])
    .agg({"EVI": "median"})
    .pipe(clean_names)
)

rainfall_adm0 = (
    pd.read_csv(
        DATA_PATH
        / "Rainfall"
        / "myanmar_chirps_rainfall_2012-01-01_2025-09-30_monthly_adm0.csv"
    )
    .assign(date=lambda df: pd.to_datetime(df["date"]))
    .set_index("date")
)

rainfall_adm1 = (
    pd.read_csv(
        DATA_PATH
        / "Rainfall"
        / "myanmar_chirps_rainfall_2012-01-01_2025-09-30_monthly_adm1.csv"
    )
    .assign(date=lambda df: pd.to_datetime(df["date"]))
    .rename(columns={"region": "adm1_name"})
    .set_index(["date", "adm1_name"])
)

## EVI and Economic Indicators

In [5]:
ntl_adm0_raw = pd.read_csv(DATA_PATH / "NTL" / "ntl_monthly_adm0_collection2.csv").drop(
    columns=["Unnamed: 0", "Unnamed: 0.1", "geometry"]
)
ntl_adm1_raw = pd.read_csv(DATA_PATH / "NTL" / "ntl_monthly_adm1_collection2.csv").drop(
    columns=["Unnamed: 0", "Unnamed: 0.1", "geometry"]
)

ntl_adm0 = (
    ntl_adm0_raw.assign(date=lambda df: pd.to_datetime(df["date"]))
    .pipe(clean_names)
    .set_index("date")
    .groupby(pd.Grouper(freq="QS"))
    .agg({"ntl_mean": "mean", "ntl_sum": "sum"})
    .assign(
        ntl_sum_lag_1=lambda df: df["ntl_sum"].shift(-1),
        ntl_sum_lag_2=lambda df: df["ntl_sum"].shift(-2),
    )
)

ntl_adm1 = (
    ntl_adm1_raw.assign(date=lambda df: pd.to_datetime(df["date"]))
    .pipe(clean_names)
    .set_index("date")
    .rename(columns={"adm1_en": "adm1_name"})
    .groupby(["adm1_name", pd.Grouper(freq="QS")])
    .agg({"ntl_mean": "mean", "ntl_sum": "sum"})
)

In [6]:
gdp_adm1_name_map = {
    "Ayeyarwaddy Region": "Ayeyarwady",
    "Bago Region": "Bago",
    "Chin State": "Chin",
    "Kachin State": "Kachin",
    "Kayah State": "Kayah",
    "Kayin State": "Kayin",
    "Magwe Region": "Magway",
    "Mandalay Region": "Mandalay",
    "Mon State": "Mon",
    "Nay Pyi Taw Council": "Nay Pyi Taw",
    "Rakhine State": "Rakhine",
    "Sagaing Region": "Sagaing",
    "Shan State": "Shan",
    "Tanintharyi Region": "Tanintharyi",
    "Yangon Region": "Yangon",
}


def preprocess_bago_and_shan(df: pd.DataFrame) -> pd.DataFrame:
    """Split the values of Bago into two equal parts and Shan into three equal parts."""
    df_bago = df.query("adm1_name == 'Bago'").copy()
    df_bago_new = pd.DataFrame(
        {
            "adm1_name": ["Bago (East)", "Bago (West)"],
            "agriculture": [df_bago["agriculture"].values[0] / 2] * 2,
            "industry": [df_bago["industry"].values[0] / 2] * 2,
            "services": [df_bago["services"].values[0] / 2] * 2,
        }
    )

    df_shan = df.query("adm1_name == 'Shan'").copy()
    df_shan_new = pd.DataFrame(
        {
            "adm1_name": ["Shan (South)", "Shan (East)", "Shan (North)"],
            "agriculture": [df_shan["agriculture"].values[0] / 3] * 3,
            "industry": [df_shan["industry"].values[0] / 3] * 3,
            "services": [df_shan["services"].values[0] / 3] * 3,
        }
    )

    df_rest = df.query("adm1_name not in ['Bago', 'Shan']").copy()

    return pd.concat([df_rest, df_bago_new, df_shan_new], ignore_index=True)


gdp_adm1 = (
    gdp_adm1_raw.pipe(clean_names)
    .assign(adm1_name=lambda df: df["adm1_name"].map(gdp_adm1_name_map))
    .pipe(preprocess_bago_and_shan)
    .assign(agriculture_pct=lambda df: df["agriculture"] / df["agriculture"].sum())
)

gdp_quarterly = (
    gdp_quarterly_raw.pipe(clean_names)
    .melt(
        id_vars=["quarter", "sub_group", "economic_activity"],
        var_name="year",
        value_name="gdp",
    )
    .assign(
        quarter_clean=lambda df: df["quarter"].str.strip(),
        year_first=lambda df: df["year"].str.split("-").str[0].str.strip(),
        year_last=lambda df: df["year"].str.split("-").str[1].str.strip(),
        # Map fiscal quarters to calendar quarters and years
        calendar_quarter=lambda df: df["quarter_clean"].map(
            {"Q1": "Q2", "Q2": "Q3", "Q3": "Q4", "Q4": "Q1"}
        ),
        year_selected=lambda df: df.apply(
            lambda row: row["year_last"]
            if row["quarter_clean"] == "Q4"
            else row["year_first"],
            axis=1,
        ),
        date=lambda df: pd.to_datetime(df["year_selected"] + df["calendar_quarter"]),
        sub_group=lambda df: df["sub_group"].str.strip(),
        economic_activity=lambda df: df["economic_activity"].str.strip(),
    )
    .query('sub_group == "Agriculture" and economic_activity == "Agriculture"')
    .set_index("date")
    .sort_index()
    .groupby(["sub_group", pd.Grouper(freq="QS")])
    .agg({"gdp": "sum"})
    .reset_index()
    .sort_values(["sub_group", "date"])
    .assign(
        gdp_lag_1=lambda df: df["gdp"].shift(1),
        gdp_lag_2=lambda df: df["gdp"].shift(2),
    )
)

gdp_quarterly_adm1 = (
    pd.DataFrame(
        [
            (region, period)
            for region in gdp_adm1["adm1_name"].unique()
            for period in gdp_quarterly["date"].unique()
        ],
        columns=["adm1_name", "date"],
    )
    .merge(
        gdp_adm1.filter(["adm1_name", "agriculture_pct"]), on="adm1_name", how="left"
    )
    .merge(gdp_quarterly, on="date", how="left")
    .rename(
        columns={
            "gdp": "gdp_total",
            "gdp_lag_1": "gdp_lag_1_total",
            "gdp_lag_2": "gdp_lag_2_total",
        }
    )
    .assign(
        gdp=lambda df: df["gdp_total"] * df["agriculture_pct"],
        gdp_lag_1=lambda df: df["gdp_lag_1_total"] * df["agriculture_pct"],
        gdp_lag_2=lambda df: df["gdp_lag_2_total"] * df["agriculture_pct"],
    )
    .set_index(["adm1_name", "date"])
    .filter(["gdp", "gdp_lag_1", "gdp_lag_2"])
)

In [7]:
evi_indicators = (
    gdp_quarterly.set_index("date")
    .join(evi_adm0, on="date", how="left")
    .join(ntl_adm0, on="date", how="left")
    .join(rainfall_adm0, on="date", how="left")
    .reset_index()
    .assign(
        evi_log=lambda df: np.log(df["evi"]),
        gdp_log=lambda df: np.log(df["gdp"]),
        ntl_sum_log=lambda df: np.log(df["ntl_sum"]),
        rainfall_mm_log=lambda df: np.log(df["rainfall_mm"]),
        evi_lag_1=lambda df: df["evi"].shift(1),
        evi_lag_2=lambda df: df["evi"].shift(2),
        rainfall_lag_1=lambda df: df["rainfall_mm"].shift(1),
        rainfall_lag_2=lambda df: df["rainfall_mm"].shift(2),
        evi_lag_1_log=lambda df: np.log(df["evi_lag_1"]),
        evi_lag_2_log=lambda df: np.log(df["evi_lag_2"]),
        rainfall_lag_1_log=lambda df: np.log(df["rainfall_lag_1"]),
        rainfall_lag_2_log=lambda df: np.log(df["rainfall_lag_2"]),
        ntl_sum_lag_1=lambda df: df["ntl_sum"].shift(1),
        is_crop_season=lambda df: df["date"].dt.month.isin([7, 8, 9, 10, 11, 12, 1, 2]),
    )
)

evi_adm0_annual = (
    evi_adm0_raw.groupby(pd.Grouper(freq="YS")).agg({"EVI": "median"}).pipe(clean_names)
)

The table below presents regression results for national median EVI and agricultural GDP. Lagged EVI (one quarter) is statistically significant across all specifications. In the full model, a 1% increase in EVI corresponds to a 1.12% increase in agricultural GDP, holding other variables constant.

In [8]:
mod_1 = smf.ols("gdp_log ~ evi_log", data=evi_indicators).fit()
mod_2 = smf.ols(
    "gdp_log ~ ntl_sum_log + evi_log + evi_lag_1_log",
    data=evi_indicators,
).fit()
mod_3 = smf.ols(
    "gdp_log ~ ntl_sum_log + evi_log + evi_lag_1_log + evi_lag_2_log",
    data=evi_indicators,
).fit()
mod_4 = smf.ols(
    "gdp_log ~ ntl_sum_log + evi_log + evi_lag_1_log + is_crop_season",
    data=evi_indicators,
).fit()

mod_4 = smf.ols(
    "gdp_log ~ ntl_sum_log + evi_log + evi_lag_1_log + is_crop_season",
    data=evi_indicators,
).fit()
mod_5 = smf.ols(
    "gdp_log ~ ntl_sum_log + evi_lag_1_log + is_crop_season",
    data=evi_indicators,
).fit()

models = Stargazer([mod_1, mod_2, mod_4, mod_5])
models.custom_columns(
    [
        "Current EVI only",
        "Current + 1 Quarter Ago",
        "Current + 1 Quarter Ago + Is Crop Season",
        "All variables",
    ],
)
models.covariate_order(
    [
        "evi_log",
        "ntl_sum_log",
        "evi_lag_1_log",
        # "evi_lag_2_log",
        "is_crop_season[T.True]",
        # "rainfall_lag_1_log",
        # "rainfall_lag_2_log",
    ]
)
models.rename_covariates(
    {
        "evi_log": "EVI (log)",
        "ntl_sum_log": "NTL Sum (log)",
        "evi_lag_1_log": "EVI 1 Quarter Ago (log)",
        # "evi_lag_2_log": "EVI 2 Quarter Ago (log)",
        "is_crop_season[T.True]": "Is Crop Season",
        # "rainfall_lag_1_log": "Rainfall 1 Quarter Ago (in mm and log)",
        # "rainfall_lag_2_log": "Rainfall 2 Quarters Ago (in mm and log)",
    }
)
models

### State-level EVI and Agricultural GDP

In [9]:
evi_indicators_adm1 = (
    gdp_quarterly_adm1.join(evi_adm1, how="left")
    .join(ntl_adm1, how="left")
    .join(rainfall_adm1, how="left")
    .reset_index()
    .sort_values(["adm1_name", "date"])
    .assign(
        evi_log=lambda df: np.log(df["evi"]),
        gdp_log=lambda df: np.log(df["gdp"]),
        rainfall_log=lambda df: np.log(df["rainfall_mm"]),
        ntl_sum_log=lambda df: np.log(df["ntl_sum"]),
        is_crop_season=lambda df: df["date"].dt.month.isin([7, 8, 9, 10, 11, 12, 1, 2]),
    )
    .groupby("adm1_name")
    .apply(
        lambda df: df.assign(
            evi_lag_1=df["evi"].shift(1),
            evi_lag_2=df["evi"].shift(2),
            rainfall_lag_1=df["rainfall_mm"].shift(1),
            rainfall_lag_2=df["rainfall_mm"].shift(2),
            evi_lag_1_log=lambda df: np.log(df["evi_lag_1"]),
            evi_lag_2_log=lambda df: np.log(df["evi_lag_2"]),
            rainfall_lag_1_log=lambda df: np.log1p(df["rainfall_lag_1"]),
            rainfall_lag_2_log=lambda df: np.log1p(df["rainfall_lag_2"]),
            ntl_sum_lag_1=df["ntl_sum"].shift(1),
            ntl_sum_lag_2=df["ntl_sum"].shift(2),
        ),
    )
    .reset_index(drop=True)
)

The table presents regression results for state-level median EVI and agricultural GDP.

In [10]:
mod_1 = smf.ols("gdp_log ~ evi_log", data=evi_indicators_adm1).fit()
mod_2 = smf.ols("gdp_log ~ ntl_sum_log + evi_log", data=evi_indicators_adm1).fit()
mod_3 = smf.ols(
    "gdp_log ~ ntl_sum_log + evi_log + evi_lag_1_log",
    data=evi_indicators_adm1,
).fit()
mod_4 = smf.ols(
    "gdp_log ~ ntl_sum_log + evi_log + evi_lag_1_log + is_crop_season",
    data=evi_indicators_adm1,
).fit()
mod_5 = smf.ols(
    "gdp_log ~ ntl_sum_log + evi_log + evi_lag_1_log + is_crop_season + C(adm1_name)",
    data=evi_indicators_adm1,
).fit()
mod_6 = smf.ols(
    "gdp_log ~ ntl_sum_log + evi_log + evi_lag_1_log + is_crop_season + rainfall_lag_1_log + C(adm1_name)",
    data=evi_indicators_adm1,
).fit()

models = Stargazer([mod_1, mod_2, mod_3, mod_4, mod_5])
models.custom_columns(
    [
        "Current EVI only",
        "Current + NTL",
        "Current + NTL + 1 Quarter Ago",
        "Current + NTL + 1 Quarter Ago + Is Crop Season",
        "All Variables + Region Fixed Effects",
        # "All Variables + Rainfall + Region Fixed Effects",
    ]
)
models.covariate_order(
    [
        "Intercept",
        "evi_log",
        "evi_lag_1_log",
        "is_crop_season[T.True]",
        "ntl_sum_log",
        # "rainfall_lag_1_log",
    ]
)
models.rename_covariates(
    {
        "evi_log": "EVI (log)",
        "evi_lag_1_log": "EVI 1 Quarter Ago (log)",
        "is_crop_season[T.True]": "Is Crop Season",
        "ntl_sum_log": "NTL Sum (log)",
        # "rainfall_lag_1_log": "Rainfall (in mm and log)",
    }
)
models

The table below shows the comparison of median EVI in 2024 and 2025 during the crop growing season (July to October)

In [11]:
evi_adm0_2024 = preprocess_evi(
    DATA_PATH
    / "EVI and Crop Land"
    / "EVI 2010-2025"
    / "Admin level 0"
    / "MIMU"
    / "myanmar_adm0_evi_stats_2024.csv"
)

evi_adm0_2025 = preprocess_evi(
    DATA_PATH
    / "EVI and Crop Land"
    / "EVI 2010-2025"
    / "Admin level 0"
    / "MIMU"
    / "myanmar_adm0_evi_stats_2025_latest.csv"
)

evi_adm0_2024_2025 = (
    pd.concat([evi_adm0_2024, evi_adm0_2025])
    .drop(columns=[".geo"])
    .assign(month=lambda df: df["date"].dt.month, year=lambda df: df["date"].dt.year)
    .set_index("date")
)

evi_adm1_2024 = pd.concat(
    [
        preprocess_evi(
            DATA_PATH
            / "EVI and Crop Land"
            / "EVI 2010-2025"
            / "Admin level 1"
            / "MIMU"
            / f"myanmar_adm1_evi_stats_2024_batch{batch}.csv"
        )
        for batch in range(1, 4)
    ]
)

evi_adm1_2025 = pd.concat(
    [
        preprocess_evi(
            DATA_PATH
            / "EVI and Crop Land"
            / "EVI 2010-2025"
            / "Admin level 1"
            / "MIMU"
            / f"myanmar_adm1_evi_stats_2025_batch{batch}_latest.csv"
        )
        for batch in range(1, 4)
    ]
)

evi_adm1_2024_2025 = (
    pd.concat([evi_adm1_2024, evi_adm1_2025])
    .drop(columns=[".geo"])
    .rename(columns={"ST": "adm1_name"})
    .assign(month=lambda df: df["date"].dt.month, year=lambda df: df["date"].dt.year)
    .set_index("date")
)

evi_adm0_2024_2025.query("month >= 7 and month <= 10").groupby(
    pd.Grouper(freq="YS")
).agg({"EVI": "median"}).pipe(clean_names)

,evi
date,
2024-01-01,0.359393
2025-01-01,0.386757


The table below shows the comparison of median EVI in 2024 and 2025 during the crop growing season (July to October) across different states in Myanmar.

In [12]:
(
    evi_adm1_2024_2025.query("month >= 7 and month <= 10")
    .groupby(["adm1_name", pd.Grouper(freq="YS")])
    .agg({"EVI": "median"})
    .pipe(clean_names)
    .reset_index()
    .pivot_table(index="adm1_name", columns="date", values="evi")
)

date,2024-01-01,2025-01-01
adm1_name,,
Ayeyarwady,0.246042,0.222713
Bago (East),0.332070,0.355437
Bago (West),0.414038,0.421869
Chin,0.466946,0.459057
Kachin,0.454188,0.464665
Kayah,0.399163,0.421751
Kayin,0.156328,0.195497
Magway,0.351606,0.374995
Mandalay,0.335932,0.386755


## Sensitivity Analysis

The following section presents sensitivity analyses to assess the robustness of the EVI-GDP relationship using national level data.

### Time-Series Cross-Validation

Using expanding window cross-validation to test model stability across different training periods.

In [13]:
min_train_size = 20
test_size = 4

cv_results = []

clean_data = evi_indicators.dropna(
    subset=["gdp_log", "evi_log", "evi_lag_1_log", "ntl_sum_log", "rainfall_lag_1_log"]
).reset_index(drop=True)

for i in range(min_train_size, len(clean_data) - test_size):
    train_cv = clean_data.iloc[:i]
    test_cv = clean_data.iloc[i : i + test_size]

    try:
        model_cv = smf.ols(
            "gdp_log ~ ntl_sum_log + evi_log + evi_lag_1_log + is_crop_season + rainfall_lag_1_log",
            data=train_cv,
        ).fit()

        predictions = model_cv.predict(test_cv)
        actuals = test_cv["gdp_log"].values

        mae = mean_absolute_error(np.exp(actuals), np.exp(predictions))
        rmse = root_mean_squared_error(np.exp(actuals), np.exp(predictions))
        mape = (
            mean_absolute_percentage_error(np.exp(actuals), np.exp(predictions)) * 100
        )

        cv_results.append(
            {
                "fold": len(cv_results) + 1,
                "train_end": train_cv["date"].iloc[-1],
                "test_start": test_cv["date"].iloc[0],
                "test_end": test_cv["date"].iloc[-1],
                "train_size": len(train_cv),
                "mae": mae,
                "rmse": rmse,
                "mape": mape,
                "evi_lag_1_coef": model_cv.params.get("evi_lag_1_log"),
            }
        )
    except:
        pass

cv_df = pd.DataFrame(cv_results).assign(
    mae=lambda df: df["mae"].round(2),
    rmse=lambda df: df["rmse"].round(2),
    mape=lambda df: df["mape"].round(2),
    evi_lag_1_coef=lambda df: df["evi_lag_1_coef"].round(4),
)
cv_df

,fold,train_end,test_start,test_end,train_size,mae,rmse,mape,evi_lag_1_coef
0,1,2017-01-01,2017-04-01,2018-01-01,20,490664.33,632271.94,18.85,0.8761
1,2,2017-04-01,2017-07-01,2018-04-01,21,447310.18,582966.94,13.95,0.8445
2,3,2017-07-01,2017-10-01,2018-07-01,22,453185.73,560006.11,13.66,0.8091
3,4,2017-10-01,2018-01-01,2018-10-01,23,353155.93,439828.87,11.32,0.7590
4,5,2018-01-01,2018-04-01,2019-01-01,24,356098.04,448500.19,11.34,0.7571
5,6,2018-04-01,2018-07-01,2019-04-01,25,367255.62,450716.29,13.35,0.7411
6,7,2018-07-01,2018-10-01,2019-07-01,26,284604.85,392170.52,9.95,0.7882
7,8,2018-10-01,2019-01-01,2019-10-01,27,275427.81,398701.18,10.11,0.7109
8,9,2019-01-01,2019-04-01,2020-01-01,28,317639.47,415638.41,11.09,0.7130
9,10,2019-04-01,2019-07-01,2020-04-01,29,271044.63,424352.79,6.01,0.7171


In [14]:
mean_mape = cv_df["mape"].mean()
mean_rmse = cv_df["rmse"].mean()

mape_line = (
    alt.Chart(cv_df)
    .mark_line(point=True, size=2)
    .encode(
        x=alt.X(
            "test_start:T",
            title="Test Period Start",
            axis=alt.Axis(format="%Y-%m"),
        ),
        y=alt.Y("mape:Q", title="MAPE (%)"),
        tooltip=alt.Tooltip(["test_start:T", "mape:Q", "rmse:Q", "mae:Q"]),
    )
)

mape_mean = (
    alt.Chart(cv_df).mark_rule(strokeDash=[5, 5], size=2).encode(y=alt.datum(mean_mape))
)

mape_chart = (mape_line + mape_mean).properties(
    width=700,
    height=250,
    title="Cross-Validation: Mean Absolute Percentage Error Over Time",
)

rmse_line = (
    alt.Chart(cv_df)
    .mark_line(point=True, size=2)
    .encode(
        x=alt.X(
            "test_start:T",
            title="Test Period Start",
            axis=alt.Axis(format="%Y-%m"),
        ),
        y=alt.Y("rmse:Q", title="RMSE"),
    )
)

rmse_mean = (
    alt.Chart(cv_df).mark_rule(strokeDash=[5, 5], size=2).encode(y=alt.datum(mean_rmse))
)

rmse_chart = (rmse_line + rmse_mean).properties(
    width=600, height=250, title="Cross-Validation: Root Mean Squared Error Over Time"
)

chart = alt.vconcat(mape_chart, rmse_chart).configure_axis(grid=True, gridOpacity=0.3)
chart

alt.VConcatChart(...)

In [15]:
mean_coef = cv_df["evi_lag_1_coef"].mean()

coef_line = (
    alt.Chart(cv_df)
    .mark_line(point=True, size=2)
    .encode(
        x=alt.X(
            "test_start:T",
            title="Test Period Start",
            axis=alt.Axis(format="%Y-%m"),
        ),
        y=alt.Y("evi_lag_1_coef:Q", title="EVI Lag 1 Coefficient"),
        tooltip=alt.Tooltip(["test_start:T", "evi_lag_1_coef:Q"]),
    )
)

coef_mean = (
    alt.Chart(cv_df).mark_rule(strokeDash=[5, 5], size=2).encode(y=alt.datum(mean_coef))
)

coef_chart = (coef_line + coef_mean).properties(
    width=600,
    height=250,
    title="Cross-Validation: EVI Lag 1 Coefficient Over Time",
)

coef_chart

alt.LayerChart(...)

### Out-of-Sample Prediction: 2023-2025

Testing the model's predictive accuracy by training on historical data and predicting recent quarters.

In [16]:
train_data = evi_indicators.query("date < '2023-01-01'").dropna()
test_data = evi_indicators.query(
    "date >= '2023-01-01' and date <= '2025-12-31'"
).dropna()

model_train = smf.ols(
    "gdp_log ~ ntl_sum_log + evi_log + evi_lag_1_log + is_crop_season",
    data=train_data,
).fit()

test_data["gdp_predicted_log"] = model_train.predict(test_data)
test_data["gdp_predicted"] = np.exp(test_data["gdp_predicted_log"])
test_data["error"] = test_data["gdp"] - test_data["gdp_predicted"]
test_data["error_pct"] = (test_data["error"] / test_data["gdp"]) * 100

prediction_results = (
    test_data[["date", "gdp", "gdp_predicted", "error", "error_pct"]]
    .round(2)
    .rename(
        columns={
            "date": "Quarter",
            "gdp": "Actual GDP",
            "gdp_predicted": "Predicted GDP",
            "error": "Error",
            "error_pct": "Error (%)",
        }
    )
)

prediction_results

,Quarter,Actual GDP,Predicted GDP,Error,Error (%)
51,2023-01-01,3551069.13,3710228.77,-159159.64,-4.48
52,2023-04-01,559112.32,404122.13,154990.19,27.72
53,2023-07-01,1669657.09,1938470.39,-268813.30,-16.10
54,2023-10-01,3897192.75,3434163.12,463029.63,11.88
55,2024-01-01,3693111.89,3511943.39,181168.50,4.91
56,2024-04-01,615023.56,606755.88,8267.68,1.34
57,2024-07-01,1836622.80,1697645.28,138977.51,7.57
58,2024-10-01,3624389.26,3556715.90,67673.35,1.87
59,2025-01-01,3434594.06,5389433.95,-1954839.89,-56.92


In [17]:
ORANGE = "#A77652"
BLUE = "#5276A7"

plot_data = (
    test_data.copy()
    .filter(["date", "gdp", "gdp_predicted", "error_pct"])
    .melt(
        id_vars=["date", "error_pct"],
        value_vars=["gdp", "gdp_predicted"],
        var_name="gdp_type",
        value_name="value",
    )
    .assign(
        gdp_type=lambda df: df["gdp_type"].map(
            {"gdp": "Actual GDP", "gdp_predicted": "Predicted GDP"}
        ),
        value=lambda df: df["value"].round(2),
        error_pct=lambda df: df["error_pct"].round(2),
    )
)

chart = (
    alt.Chart(plot_data)
    .mark_line(point=True, strokeWidth=2)
    .encode(
        x=alt.X("date:T", title="Quarter"),
        y=alt.Y("value:Q", title="Agricultural GDP"),
        color=alt.Color(
            "gdp_type:N",
            scale=alt.Scale(
                domain=["Actual GDP", "Predicted GDP"], range=[BLUE, ORANGE]
            ),
            legend=alt.Legend(title=None),
        ),
        strokeDash=alt.StrokeDash(
            "gdp_type:N",
            scale=alt.Scale(
                domain=["Actual GDP", "Predicted GDP"], range=[[1, 0], [5, 5]]
            ),
        ),
        tooltip=alt.Tooltip(
            ["date:T", "gdp_type:N", "value:Q", "error_pct:Q"],
        ),
    )
    .properties(width=600, height=300, title="Out-of-Sample Predictions: 2023-2024")
)

chart

alt.Chart(...)

### Variable Exclusion/Inclusion Tests

Tests the impact of including or excluding key variables to assess their marginal contribution to the model.

In [18]:
base_vars = [
    "ntl_sum_log",
    "evi_log",
    "evi_lag_1_log",
    "is_crop_season",
]

exclusion_models = {}
exclusion_models["Full Model"] = smf.ols(
    "gdp_log ~ ntl_sum_log + evi_log + evi_lag_1_log + is_crop_season",
    data=evi_indicators,
).fit()

for var_to_exclude in base_vars:
    remaining_vars = [v for v in base_vars if v != var_to_exclude]
    formula = "gdp_log ~ " + " + ".join(remaining_vars)
    model_name = f"Exclude {var_to_exclude}"
    exclusion_models[model_name] = smf.ols(formula, data=evi_indicators).fit()

exclusion_models["Remove rainfall and nighttime lights"] = smf.ols(
    "gdp_log ~ evi_log + evi_lag_1_log + is_crop_season",
    data=evi_indicators,
).fit()

exclusion_stargazer = Stargazer(list(exclusion_models.values()))
exclusion_stargazer.custom_columns(
    list(exclusion_models.keys()), [1] * len(exclusion_models)
)
exclusion_stargazer.rename_covariates(
    {
        "evi_log": "EVI (log)",
        "ntl_sum_log": "NTL Sum (log)",
        "evi_lag_1_log": "EVI 1 Quarter Ago (log)",
        "is_crop_season[T.True]": "Is Crop Season",
        "rainfall_lag_1_log": "Rainfall 1 Quarter Ago (log)",
    }
)
exclusion_stargazer